In [27]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


In [39]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'your openapi key here'
embeddings = OpenAIEmbeddings()


In [40]:
# Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500)

In [41]:
!pip install openai

In [42]:
#Load Data

In [43]:
loaders = UnstructuredURLLoader(urls=[
 "https://www.bing.com/search?pglt=2083&q=modi+came+to+united+states&cvid=3d1ee12d144e4328a04b269da631ab6a&gs_lcrp=EgRlZGdlKgYIABBFGDkyBggAEEUYOTIGCAEQABhAMgYIAhAAGEAyBggDEAAYQDIGCAQQABhAMgYIBRAAGEAyBggGEAAYQDIGCAcQABhAMgYICBAAGEAyCAgJEOkHGPxV0gEINDQ1MmowajGoAgCwAgA&FORM=ANNAB1&PC=HCTS",
    "https://www.bing.com/search?pglt=2083&q=sanam+teri+kasam+rerelease&cvid=668f3e69ea8a40dfa061cf8cb3923966&gs_lcrp=EgRlZGdlKgYIABBFGDkyBggAEEUYOTIGCAEQABhAMgYIAhAAGEAyBggDEAAYQDIGCAQQABhAMgYIBRAAGEAyBggGEAAYQDIGCAcQABhAMgYICBAAGEAyCAgJEOkHGPxV0gEINjg2OGowajGoAgiwAgE&FORM=ANNAB1&PC=HCTS",
    "https://www.bing.com/search?q=data%20analyst%20skills&qs=n&form=QBRE&sp=-1&ghc=1&lq=0&pq=data%20analyst%20skill&sc=12-18&sk=&cvid=8D79DF1CE7AE4CFFBE8E7F1DB1ACC923&ghsh=0&ghacc=0&ghpl="
])
data = loaders.load() 
len(data)

3

In [44]:
#split data to create chunks

In [45]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [46]:
len(docs)

15

In [47]:
docs[9]

Document(metadata={'source': 'https://www.bing.com/search?pglt=2083&q=sanam+teri+kasam+rerelease&cvid=668f3e69ea8a40dfa061cf8cb3923966&gs_lcrp=EgRlZGdlKgYIABBFGDkyBggAEEUYOTIGCAEQABhAMgYIAhAAGEAyBggDEAAYQDIGCAQQABhAMgYIBRAAGEAyBggGEAAYQDIGCAcQABhAMgYICBAAGEAyCAgJEOkHGPxV0gEINjg2OGowajGoAgiwAgE&FORM=ANNAB1&PC=HCTS'}, page_content="indiaforums.com\n\nhttps://www.indiaforums.com › article\n\n'Sanam Teri Kasam 2' officially announced with Harshvardhan …\n\nSep 10, 2024 · Harshvardhan Rane, who captured hearts with his performance in the original film, is all set to return as the lead in Sanam Teri Kasam 2. The project is backed by Soham Rockstar Entertainment, with producer Deepak Mukut confirming that the sequel's story has been finalized.\n\nRefresh")

In [48]:
#(3) Create embeddings for these chunks and save them to FAISS index


In [51]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your ope*********here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [25]:
!pip install -U langchain langchain-openai openai

In [ ]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [ ]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)


In [ ]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [52]:
#Retrieve similar embeddings for a given question and call LLM to retrieve final answer

SyntaxError: invalid syntax (3831585934.py, line 1)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

In [ ]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)